# Algorithmic Trading Strategy Analysis

This notebook demonstrates the implementation and analysis of various algorithmic trading strategies including:
1. **Trend-Following Strategies**: EMA-ADX crossover, MACD, and FX trend ratio
2. **Mean-Reversion Strategies**: Z-score based signals and Ornstein-Uhlenbeck process
3. **Risk Management**: VaR, Expected Shortfall, and position sizing
4. **Performance Analysis**: Sharpe ratio, maximum drawdown, and trade analysis

---

In [ ]:
# Import necessary libraries
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
from pathlib import Path
import os

# Add project root to path - handle both notebook and script execution
notebook_dir = Path.cwd()
if notebook_dir.name == 'notebooks':
    # Running from notebooks directory
    project_root = notebook_dir.parent
else:
    # Running from project root
    project_root = notebook_dir

# Import simplified trading modules
sys.path.insert(0, str(project_root))
from simple_trading import data_manager, SimpleTradingStrategy, SimpleBacktester

# Setup
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All modules imported successfully!")
print(f"Data manager: {type(data_manager).__name__}")
print(f"Project root: {project_root}")

ImportError: attempted relative import beyond top-level package

## 📊 Data Acquisition and Preparation

Let's start by fetching historical market data for our analysis.

In [ ]:
# Define analysis parameters
SYMBOLS = ['AAPL', 'GOOGL', 'MSFT', 'SPY']
START_DATE = datetime(2022, 1, 1)
END_DATE = datetime(2024, 1, 1)
FREQUENCY = '1d'
INITIAL_CAPITAL = 100000

print(f"Analysis Period: {START_DATE.date()} to {END_DATE.date()}")
print(f"Symbols: {', '.join(SYMBOLS)}")
print(f"Initial Capital: ${INITIAL_CAPITAL:,}")

In [ ]:
# Fetch data for all symbols
data_dict = {}

for symbol in SYMBOLS:
    print(f"Fetching data for {symbol}...")
    data = data_manager.get_historical_data(
        symbol=symbol,
        start_date=START_DATE,
        end_date=END_DATE,
        frequency=FREQUENCY
    )
    
    if not data.empty:
        data_dict[symbol] = data
        print(f"  ✅ Retrieved {len(data)} records")
    else:
        print(f"  ❌ No data found")

print(f"\nSuccessfully loaded data for {len(data_dict)} symbols")

In [ ]:
# Display data summary
if data_dict:
    sample_symbol = list(data_dict.keys())[0]
    sample_data = data_dict[sample_symbol]
    
    print(f"Sample data for {sample_symbol}:")
    print(sample_data.head())
    print(f"\nData shape: {sample_data.shape}")
    print(f"Date range: {sample_data.index[0].date()} to {sample_data.index[-1].date()}")

## 📈 Strategy 1: EMA-ADX Trend Following

### Mathematical Foundation

**Exponential Moving Average (EMA):**
$$EMA_t = \alpha \cdot P_t + (1-\alpha) \cdot EMA_{t-1}$$
where $\alpha = \frac{2}{n+1}$ and $n$ is the period.

**Average Directional Index (ADX):**
- Measures trend strength from 0-100
- ADX > 25 indicates strong trend
- Based on Directional Indicators (+DI, -DI)

**Signal Logic:**
- **BUY**: Fast EMA crosses above Slow EMA AND ADX > threshold
- **SELL**: Fast EMA crosses below Slow EMA AND ADX > threshold

In [ ]:
# Initialize simplified strategies for demonstration
class EMAAdxStrategy(SimpleTradingStrategy):
    """Simplified EMA-ADX Strategy"""
    
    def __init__(self, config):
        super().__init__(config)
        self.ema_fast = config.get('ema_fast', 12)
        self.ema_slow = config.get('ema_slow', 26)
    
    def calculate_indicators(self, data):
        """Calculate EMA indicators"""
        indicators = data.copy()
        indicators['ema_fast'] = self.calculate_ema(data['close'], self.ema_fast)
        indicators['ema_slow'] = self.calculate_ema(data['close'], self.ema_slow)
        indicators['signal'] = 0
        
        # Simple crossover signals
        indicators.loc[indicators['ema_fast'] > indicators['ema_slow'], 'signal'] = 1
        indicators.loc[indicators['ema_fast'] < indicators['ema_slow'], 'signal'] = -1
        
        return indicators

class ZScoreMeanReversionStrategy(SimpleTradingStrategy):
    """Simplified Z-Score Strategy"""
    
    def __init__(self, config):
        super().__init__(config)
        self.lookback_window = config.get('lookback_window', 20)
        self.entry_threshold = config.get('entry_threshold', 2.0)
    
    def calculate_indicators(self, data):
        """Calculate Z-Score indicators"""
        indicators = data.copy()
        indicators['rolling_mean'] = data['close'].rolling(self.lookback_window).mean()
        indicators['rolling_std'] = data['close'].rolling(self.lookback_window).std()
        indicators['zscore'] = (data['close'] - indicators['rolling_mean']) / indicators['rolling_std']
        
        # Bollinger Bands
        indicators['bb_upper'] = indicators['rolling_mean'] + (2 * indicators['rolling_std'])
        indicators['bb_lower'] = indicators['rolling_mean'] - (2 * indicators['rolling_std'])
        
        # Simple mean reversion signals
        indicators['signal'] = 0
        indicators.loc[indicators['zscore'] < -self.entry_threshold, 'signal'] = 1  # Buy when oversold
        indicators.loc[indicators['zscore'] > self.entry_threshold, 'signal'] = -1  # Sell when overbought
        
        return indicators

# Create strategy instances with configurations
ema_config = {
    'ema_fast': 12,
    'ema_slow': 26,
    'adx_period': 14,
    'adx_threshold': 25
}

zscore_config = {
    'lookback_window': 20,
    'entry_threshold': 2.0,
    'exit_threshold': 0.5,
    'use_ou_process': False
}

print("Strategy classes created successfully!")
print(f"EMA Strategy: Fast={ema_config['ema_fast']}, Slow={ema_config['ema_slow']}")
print(f"Z-Score Strategy: Window={zscore_config['lookback_window']}, Threshold=±{zscore_config['entry_threshold']}")

In [ ]:
# Analyze EMA strategy on AAPL
if 'AAPL' in data_dict:
    aapl_data = data_dict['AAPL'].copy()
    
    # Create strategy instance
    ema_strategy = EMAAdxStrategy(ema_config)
    
    # Calculate indicators
    aapl_indicators = ema_strategy.calculate_indicators(aapl_data)
    
    # Count signals
    buy_signals = aapl_indicators[aapl_indicators['signal'] == 1]
    sell_signals = aapl_indicators[aapl_indicators['signal'] == -1]
    
    print(f"Generated {len(buy_signals)} BUY signals and {len(sell_signals)} SELL signals for AAPL")
    
    # Display sample signals
    if len(buy_signals) > 0:
        print("\\nSample BUY signals:")
        for i, (date, row) in enumerate(buy_signals.head().iterrows()):
            print(f"  {i+1}. {date.date()}: BUY at ${row['close']:.2f}")
    
    if len(sell_signals) > 0:
        print("\\nSample SELL signals:")
        for i, (date, row) in enumerate(sell_signals.head().iterrows()):
            print(f"  {i+1}. {date.date()}: SELL at ${row['close']:.2f}")
else:
    print("⚠️ AAPL data not available")

In [ ]:
# Visualize EMA-ADX indicators
if 'AAPL' in data_dict and len(aapl_indicators) > 0:
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # Price and EMAs
    axes[0].plot(aapl_indicators.index, aapl_indicators['close'], label='Close Price', alpha=0.7)
    axes[0].plot(aapl_indicators.index, aapl_indicators['ema_fast'], label=f'EMA {ema_config["ema_fast"]}', linewidth=2)
    axes[0].plot(aapl_indicators.index, aapl_indicators['ema_slow'], label=f'EMA {ema_config["ema_slow"]}', linewidth=2)
    
    # Add signal markers
    buy_signals = [s for s in ema_signals if s.signal_type.name == 'BUY']
    sell_signals = [s for s in ema_signals if s.signal_type.name == 'SELL']
    
    if buy_signals:
        buy_dates = [s.timestamp for s in buy_signals]
        buy_prices = [aapl_indicators.loc[s.timestamp]['close'] for s in buy_signals if s.timestamp in aapl_indicators.index]
        axes[0].scatter(buy_dates[:len(buy_prices)], buy_prices, color='green', marker='^', s=100, label='Buy Signal', zorder=5)
    
    if sell_signals:
        sell_dates = [s.timestamp for s in sell_signals]
        sell_prices = [aapl_indicators.loc[s.timestamp]['close'] for s in sell_signals if s.timestamp in aapl_indicators.index]
        axes[0].scatter(sell_dates[:len(sell_prices)], sell_prices, color='red', marker='v', s=100, label='Sell Signal', zorder=5)
    
    axes[0].set_title('AAPL: EMA Crossover Strategy', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Price ($)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # ADX
    axes[1].plot(aapl_indicators.index, aapl_indicators['adx'], label='ADX', color='purple', linewidth=2)
    axes[1].axhline(y=ema_config['adx_threshold'], color='red', linestyle='--', alpha=0.7, label=f'Threshold ({ema_config["adx_threshold"]})')
    axes[1].fill_between(aapl_indicators.index, 0, ema_config['adx_threshold'], alpha=0.1, color='red', label='Weak Trend Zone')
    axes[1].set_title('Average Directional Index (ADX)', fontsize=12)
    axes[1].set_ylabel('ADX')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Directional Indicators
    if 'di_plus' in aapl_indicators.columns:
        axes[2].plot(aapl_indicators.index, aapl_indicators['di_plus'], label='+DI', color='green')
        axes[2].plot(aapl_indicators.index, aapl_indicators['di_minus'], label='-DI', color='red')
        axes[2].set_title('Directional Indicators', fontsize=12)
        axes[2].set_ylabel('DI')
        axes[2].legend()
        axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No data available for visualization")

## 📉 Strategy 2: Z-Score Mean Reversion

### Mathematical Foundation

**Z-Score Formula:**
$$Z_t = \frac{P_t - \mu_t}{\sigma_t}$$

where:
- $P_t$ = Current price
- $\mu_t$ = Rolling mean over lookback window
- $\sigma_t$ = Rolling standard deviation over lookback window

**Signal Logic:**
- **BUY**: $Z_t < -threshold$ (oversold, expect upward reversion)
- **SELL**: $Z_t > +threshold$ (overbought, expect downward reversion)
- **EXIT**: $|Z_t| < exit\_threshold$ (return to mean)

In [ ]:
# Initialize Z-Score strategy
zscore_config = {
    'lookback_window': 20,
    'entry_threshold': 2.0,
    'exit_threshold': 0.5,
    'use_ou_process': False
}

zscore_strategy = ZScoreMeanReversionStrategy(zscore_config)
print("Z-Score Mean Reversion Strategy Configuration:")
print(f"  Lookback Window: {zscore_config['lookback_window']} periods")
print(f"  Entry Threshold: ±{zscore_config['entry_threshold']}")
print(f"  Exit Threshold: ±{zscore_config['exit_threshold']}")

In [ ]:
# Generate Z-Score signals for AAPL
zscore_strategy.reset()  # Reset strategy state

if 'AAPL' in data_dict:
    aapl_data.attrs['symbol'] = 'AAPL'
    
    # Calculate indicators
    zscore_indicators = zscore_strategy.calculate_indicators(aapl_data)
    
    # Generate signals
    zscore_signals = zscore_strategy.generate_signals(aapl_data)
    
    print(f"Generated {len(zscore_signals)} Z-Score signals for AAPL")
    
    # Display sample signals
    if zscore_signals:
        print("\nSample signals:")
        for i, signal in enumerate(zscore_signals[:5]):
            print(f"  {i+1}. {signal.timestamp.date()}: {signal.signal_type.name} (strength: {signal.strength:.2f})")

In [ ]:
# Visualize Z-Score analysis
if len(zscore_indicators) > 0:
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # Price with Bollinger Bands
    axes[0].plot(zscore_indicators.index, zscore_indicators['close'], label='Close Price', alpha=0.8)
    axes[0].plot(zscore_indicators.index, zscore_indicators['rolling_mean'], label='Rolling Mean (20)', linewidth=2)
    axes[0].fill_between(zscore_indicators.index, 
                        zscore_indicators['bb_lower'], 
                        zscore_indicators['bb_upper'], 
                        alpha=0.2, label='Bollinger Bands (±2σ)')
    
    # Add signal markers
    buy_signals = [s for s in zscore_signals if s.signal_type.name == 'BUY']
    sell_signals = [s for s in zscore_signals if s.signal_type.name == 'SELL']
    
    if buy_signals:
        buy_dates = [s.timestamp for s in buy_signals]
        buy_prices = [zscore_indicators.loc[s.timestamp]['close'] for s in buy_signals if s.timestamp in zscore_indicators.index]
        axes[0].scatter(buy_dates[:len(buy_prices)], buy_prices, color='green', marker='^', s=100, label='Buy Signal', zorder=5)
    
    if sell_signals:
        sell_dates = [s.timestamp for s in sell_signals]
        sell_prices = [zscore_indicators.loc[s.timestamp]['close'] for s in sell_signals if s.timestamp in zscore_indicators.index]
        axes[0].scatter(sell_dates[:len(sell_prices)], sell_prices, color='red', marker='v', s=100, label='Sell Signal', zorder=5)
    
    axes[0].set_title('AAPL: Price with Bollinger Bands', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Price ($)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Z-Score
    axes[1].plot(zscore_indicators.index, zscore_indicators['zscore'], label='Z-Score', color='purple', linewidth=2)
    axes[1].axhline(y=zscore_config['entry_threshold'], color='red', linestyle='--', alpha=0.7, label=f'Entry Threshold (±{zscore_config["entry_threshold"]})')
    axes[1].axhline(y=-zscore_config['entry_threshold'], color='red', linestyle='--', alpha=0.7)
    axes[1].axhline(y=zscore_config['exit_threshold'], color='green', linestyle=':', alpha=0.7, label=f'Exit Threshold (±{zscore_config["exit_threshold"]})')
    axes[1].axhline(y=-zscore_config['exit_threshold'], color='green', linestyle=':', alpha=0.7)
    axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.5, linewidth=1)
    
    # Color zones
    axes[1].fill_between(zscore_indicators.index, -zscore_config['entry_threshold'], -4, alpha=0.1, color='green', label='Oversold Zone')
    axes[1].fill_between(zscore_indicators.index, zscore_config['entry_threshold'], 4, alpha=0.1, color='red', label='Overbought Zone')
    
    axes[1].set_title('Z-Score Analysis', fontsize=12)
    axes[1].set_ylabel('Z-Score')
    axes[1].set_ylim(-4, 4)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Rolling Standard Deviation
    axes[2].plot(zscore_indicators.index, zscore_indicators['rolling_std'], label='Rolling Std (20)', color='orange')
    axes[2].set_title('Rolling Standard Deviation', fontsize=12)
    axes[2].set_ylabel('Standard Deviation')
    axes[2].set_xlabel('Date')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No indicator data available for visualization")

## 🌍 Strategy 3: FX Trend (Price Ratio Method)

### Mathematical Foundation

This strategy uses a simple but effective approach for trend detection:

**Price Ratio:**
$$Ratio_t = \frac{P_{short}(t)}{P_{long\_avg}(t)}$$

where:
- $P_{short}(t)$ = Short-term price (e.g., 30-second average)
- $P_{long\_avg}(t)$ = Long-term average price (e.g., 5-minute average)

**Signal Logic:**
- **Uptrend**: $Ratio > 1 + threshold$
- **Downtrend**: $Ratio < 1 - threshold$
- **No Trend**: $1 - band < Ratio < 1 + band$

In [ ]:
# Initialize FX Trend strategy
fx_config = {
    'short_interval': '30min',  # Using 30min for daily data demonstration
    'long_interval': '2h',      # Using 2h for daily data demonstration
    'trend_threshold': 0.02,
    'no_trend_band': 0.01
}

fx_strategy = FXTrendStrategy(fx_config)
print("FX Trend Strategy Configuration:")
print(f"  Short Interval: {fx_config['short_interval']}")
print(f"  Long Interval: {fx_config['long_interval']}")
print(f"  Trend Threshold: ±{fx_config['trend_threshold']:.1%}")
print(f"  No-Trend Band: ±{fx_config['no_trend_band']:.1%}")

## 🏆 Comprehensive Backtesting

Now let's run comprehensive backtests for all strategies and compare their performance.

In [ ]:
# Initialize backtesting engine
backtest_config = {
    'initial_capital': INITIAL_CAPITAL,
    'commission': 0.001,  # 0.1% commission
    'slippage': 0.0005    # 0.05% slippage
}

engine = BacktestEngine(backtest_config)
print("Backtesting Engine Configuration:")
print(f"  Initial Capital: ${backtest_config['initial_capital']:,}")
print(f"  Commission: {backtest_config['commission']:.1%}")
print(f"  Slippage: {backtest_config['slippage']:.2%}")

In [ ]:
# Run backtests for all strategies on AAPL
if 'AAPL' in data_dict:
    symbol = 'AAPL'
    data = data_dict[symbol]
    
    strategies = {
        'EMA-ADX': EMAAdxStrategy(ema_config),
        'Z-Score': ZScoreMeanReversionStrategy(zscore_config)
    }
    
    results = {}
    
    for name, strategy in strategies.items():
        print(f"\n🚀 Running backtest for {name} strategy on {symbol}...")
        
        try:
            result = engine.run_backtest(
                strategy=strategy,
                data=data,
                symbol=symbol,
                start_date=START_DATE,
                end_date=END_DATE
            )
            
            results[name] = result
            
            print(f"  ✅ Completed: {result.num_trades} trades, {result.total_return:.2%} return")
            
        except Exception as e:
            print(f"  ❌ Error: {e}")
    
    print(f"\n📊 Completed backtests for {len(results)} strategies")
else:
    print("⚠️ No AAPL data available for backtesting")
    results = {}

In [ ]:
# Display detailed results
if results:
    print("\n" + "="*80)
    print("📈 BACKTESTING RESULTS SUMMARY")
    print("="*80)
    
    for name, result in results.items():
        print(f"\n🔍 {name} Strategy:")
        print(f"  Symbol: {result.symbol}")
        print(f"  Period: {result.start_date.date()} to {result.end_date.date()}")
        print(f"  Initial Capital: ${result.initial_capital:,.2f}")
        
        if result.portfolio_values:
            final_value = result.portfolio_values[-1]
            print(f"  Final Value: ${final_value:,.2f}")
        
        print(f"  Total Return: {result.total_return:.2%}")
        print(f"  Annualized Return: {result.annualized_return:.2%}")
        print(f"  Volatility: {result.volatility:.2%}")
        print(f"  Sharpe Ratio: {result.sharpe_ratio:.2f}")
        print(f"  Max Drawdown: {result.max_drawdown:.2%}")
        print(f"  Win Rate: {result.win_rate:.1%}")
        print(f"  Number of Trades: {result.num_trades}")
        print(f"  Avg Trade Return: {result.avg_trade_return:.2%}")
        print("-" * 50)

In [ ]:
# Create performance comparison table
if results:
    comparison_data = []
    
    for name, result in results.items():
        comparison_data.append({
            'Strategy': name,
            'Total Return': f"{result.total_return:.2%}",
            'Annualized Return': f"{result.annualized_return:.2%}",
            'Volatility': f"{result.volatility:.2%}",
            'Sharpe Ratio': f"{result.sharpe_ratio:.2f}",
            'Max Drawdown': f"{result.max_drawdown:.2%}",
            'Win Rate': f"{result.win_rate:.1%}",
            'Trades': result.num_trades,
            'Avg Trade': f"{result.avg_trade_return:.2%}"
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    
    print("\n📊 STRATEGY COMPARISON TABLE")
    print("=" * 120)
    print(comparison_df.to_string(index=False))
    print("=" * 120)

In [ ]:
# Visualize backtest results
if results:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Portfolio Values
    for name, result in results.items():
        if result.portfolio_values and result.timestamps:
            axes[0, 0].plot(result.timestamps, result.portfolio_values, 
                           label=f'{name}', linewidth=2, alpha=0.8)
    
    axes[0, 0].axhline(y=INITIAL_CAPITAL, color='black', linestyle='--', alpha=0.5, label='Initial Capital')
    axes[0, 0].set_title('Portfolio Value Over Time', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Portfolio Value ($)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Cumulative Returns
    for name, result in results.items():
        if result.returns and result.timestamps[1:]:
            cumulative_returns = np.cumprod(1 + np.array(result.returns[1:])) - 1
            axes[0, 1].plot(result.timestamps[1:len(cumulative_returns)+1], cumulative_returns, 
                           label=f'{name}', linewidth=2, alpha=0.8)
    
    axes[0, 1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
    axes[0, 1].set_title('Cumulative Returns', fontsize=14, fontweight='bold')
    axes[0, 1].set_ylabel('Cumulative Return')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Performance Metrics Comparison
    metrics = ['Total Return', 'Sharpe Ratio', 'Max Drawdown', 'Win Rate']
    strategy_names = list(results.keys())
    
    total_returns = [results[name].total_return * 100 for name in strategy_names]
    sharpe_ratios = [results[name].sharpe_ratio for name in strategy_names]
    max_drawdowns = [abs(results[name].max_drawdown) * 100 for name in strategy_names]
    win_rates = [results[name].win_rate * 100 for name in strategy_names]
    
    x = np.arange(len(strategy_names))
    width = 0.2
    
    axes[1, 0].bar(x - 1.5*width, total_returns, width, label='Total Return (%)', alpha=0.8)
    axes[1, 0].bar(x - 0.5*width, [s*10 for s in sharpe_ratios], width, label='Sharpe Ratio (×10)', alpha=0.8)
    axes[1, 0].bar(x + 0.5*width, max_drawdowns, width, label='Max Drawdown (%)', alpha=0.8)
    axes[1, 0].bar(x + 1.5*width, win_rates, width, label='Win Rate (%)', alpha=0.8)
    
    axes[1, 0].set_title('Performance Metrics Comparison', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('Value')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(strategy_names)
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Trade Count Comparison
    trade_counts = [results[name].num_trades for name in strategy_names]
    
    axes[1, 1].bar(strategy_names, trade_counts, alpha=0.8, color=['skyblue', 'lightcoral'])
    axes[1, 1].set_title('Number of Trades', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('Trade Count')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Add value labels on bars
    for i, v in enumerate(trade_counts):
        axes[1, 1].text(i, v + max(trade_counts)*0.01, str(v), ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No results available for visualization")

## ⚠️ Risk Analysis

Risk management is crucial for algorithmic trading. Let's analyze the risk metrics of our strategies.

In [ ]:
# Initialize risk manager
risk_manager = RiskManager()

print("Risk Manager Configuration:")
print(f"  Max Position Size: {risk_manager.max_position_size:.1%}")
print(f"  Stop Loss: {risk_manager.stop_loss:.1%}")
print(f"  Max Portfolio Risk: {risk_manager.max_portfolio_risk:.1%}")
print(f"  VaR Confidence: {risk_manager.var_confidence:.1%}")

In [ ]:
# Calculate risk metrics for each strategy
if results:
    risk_reports = {}
    
    for name, result in results.items():
        if result.returns:
            returns_series = pd.Series(result.returns[1:])  # Skip first return (always 0)
            
            if not returns_series.empty:
                # Generate risk report
                risk_report = risk_manager.generate_risk_report(
                    portfolio_returns=returns_series,
                    positions={},  # Simplified for demo
                    portfolio_value=result.portfolio_values[-1] if result.portfolio_values else INITIAL_CAPITAL
                )
                
                risk_reports[name] = risk_report
    
    print(f"\n📊 Generated risk reports for {len(risk_reports)} strategies")
else:
    risk_reports = {}

In [ ]:
# Display risk analysis
if risk_reports:
    print("\n" + "="*80)
    print("⚠️  RISK ANALYSIS SUMMARY")
    print("="*80)
    
    risk_comparison = []
    
    for name, report in risk_reports.items():
        print(f"\n📊 {name} Strategy Risk Metrics:")
        print(f"  Portfolio Value: ${report.get('portfolio_value', 0):,.2f}")
        print(f"  Annualized Volatility: {report.get('volatility_annualized', 0):.2%}")
        print(f"  VaR (95%, 1-day): {report.get('var_95_1day', 0):.2%}")
        print(f"  Expected Shortfall (95%): {report.get('expected_shortfall_95', 0):.2%}")
        print(f"  Maximum Drawdown: {report.get('max_drawdown', 0):.2%}")
        print(f"  Current Drawdown: {report.get('current_drawdown', 0):.2%}")
        
        # Add to comparison
        risk_comparison.append({
            'Strategy': name,
            'Volatility': f"{report.get('volatility_annualized', 0):.2%}",
            'VaR (95%)': f"{report.get('var_95_1day', 0):.2%}",
            'Expected Shortfall': f"{report.get('expected_shortfall_95', 0):.2%}",
            'Max Drawdown': f"{report.get('max_drawdown', 0):.2%}",
            'Current Drawdown': f"{report.get('current_drawdown', 0):.2%}"
        })
        
        # Risk warnings
        if report.get('warnings'):
            print(f"  ⚠️  Risk Warnings:")
            for warning in report['warnings']:
                print(f"    • {warning}")
        else:
            print(f"  ✅ No risk warnings")
        
        print("-" * 50)
    
    # Risk comparison table
    if risk_comparison:
        risk_df = pd.DataFrame(risk_comparison)
        print("\n📊 RISK METRICS COMPARISON")
        print("=" * 100)
        print(risk_df.to_string(index=False))
        print("=" * 100)
else:
    print("⚠️ No risk reports available")

In [ ]:
# Visualize risk metrics
if results:
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Return distribution for each strategy
    for i, (name, result) in enumerate(results.items()):
        if result.returns:
            returns = np.array(result.returns[1:])  # Skip first return
            if len(returns) > 0:
                axes[0, i].hist(returns, bins=50, alpha=0.7, density=True, 
                               label=f'{name}', color=plt.cm.Set1(i))
                axes[0, i].axvline(np.mean(returns), color='red', linestyle='--', 
                                  label=f'Mean: {np.mean(returns):.3f}', alpha=0.8)
                axes[0, i].axvline(np.percentile(returns, 5), color='orange', linestyle=':', 
                                  label=f'VaR (95%): {np.percentile(returns, 5):.3f}', alpha=0.8)
                axes[0, i].set_title(f'{name} Return Distribution', fontweight='bold')
                axes[0, i].set_xlabel('Daily Return')
                axes[0, i].set_ylabel('Density')
                axes[0, i].legend()
                axes[0, i].grid(True, alpha=0.3)
    
    # Drawdown analysis
    for name, result in results.items():
        if result.portfolio_values:
            portfolio_series = pd.Series(result.portfolio_values)
            running_max = portfolio_series.expanding().max()
            drawdowns = (portfolio_series - running_max) / running_max
            
            axes[1, 0].plot(result.timestamps, drawdowns * 100, 
                           label=f'{name}', linewidth=2, alpha=0.8)
    
    axes[1, 0].fill_between(axes[1, 0].get_xlim(), -risk_manager.max_drawdown*100, 0, 
                           alpha=0.1, color='red', label=f'Max DD Limit ({risk_manager.max_drawdown:.1%})')
    axes[1, 0].set_title('Drawdown Over Time', fontweight='bold')
    axes[1, 0].set_ylabel('Drawdown (%)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Rolling volatility (if we have enough data points)
    window = min(30, len(result.returns) // 4) if results else 30
    
    for name, result in results.items():
        if result.returns and len(result.returns) > window:
            returns_series = pd.Series(result.returns[1:])
            rolling_vol = returns_series.rolling(window=window).std() * np.sqrt(252) * 100  # Annualized %
            
            axes[1, 1].plot(result.timestamps[window:len(rolling_vol)+window], rolling_vol.iloc[window-1:], 
                           label=f'{name}', linewidth=2, alpha=0.8)
    
    axes[1, 1].set_title(f'Rolling Volatility ({window}-day window)', fontweight='bold')
    axes[1, 1].set_ylabel('Annualized Volatility (%)')
    axes[1, 1].set_xlabel('Date')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No data available for risk visualization")

## 🎯 Conclusions and Key Insights

### Strategy Performance Summary

Based on our analysis, here are the key findings:

1. **EMA-ADX Trend Following Strategy**:
   - Best suited for trending markets
   - ADX filter helps avoid false signals in sideways markets
   - May have fewer trades but higher conviction signals

2. **Z-Score Mean Reversion Strategy**:
   - Effective in range-bound or mean-reverting markets
   - Higher trade frequency than trend-following strategies
   - Risk of prolonged trending moves against positions

3. **FX Trend Strategy**:
   - Simple yet effective approach for trend detection
   - Particularly useful for FX markets with shorter timeframes
   - Requires high-frequency data for optimal performance

### Risk Management Insights

- **VaR Analysis**: Provides daily risk estimates for position sizing
- **Drawdown Monitoring**: Critical for capital preservation
- **Volatility Analysis**: Helps adjust position sizes based on market conditions

### Recommendations for Live Trading

1. **Strategy Combination**: Consider combining trend-following and mean-reversion strategies
2. **Market Regime Detection**: Adapt strategy selection based on market conditions
3. **Risk Controls**: Implement strict stop-losses and position sizing rules
4. **Paper Trading**: Test extensively with paper trading before live deployment
5. **Continuous Monitoring**: Regular performance and risk assessment

### Next Steps

- **Live Data Integration**: Connect to real-time data feeds
- **Broker API Integration**: Implement order execution with proper error handling
- **Database Storage**: Store trades and performance data for analysis
- **Alert System**: Set up monitoring and alert systems
- **Strategy Optimization**: Continuously optimize parameters based on performance

In [ ]:
# Final summary statistics
if results:
    print("\n" + "="*80)
    print("🎯 FINAL ANALYSIS SUMMARY")
    print("="*80)
    
    best_return = max(results.items(), key=lambda x: x[1].total_return)
    best_sharpe = max(results.items(), key=lambda x: x[1].sharpe_ratio)
    lowest_drawdown = min(results.items(), key=lambda x: abs(x[1].max_drawdown))
    
    print(f"🏆 Best Total Return: {best_return[0]} ({best_return[1].total_return:.2%})")
    print(f"📊 Best Sharpe Ratio: {best_sharpe[0]} ({best_sharpe[1].sharpe_ratio:.2f})")
    print(f"🛡️  Lowest Max Drawdown: {lowest_drawdown[0]} ({lowest_drawdown[1].max_drawdown:.2%})")
    
    # Overall portfolio stats if we had equal allocation
    avg_return = np.mean([r.total_return for r in results.values()])
    avg_sharpe = np.mean([r.sharpe_ratio for r in results.values()])
    avg_drawdown = np.mean([r.max_drawdown for r in results.values()])
    
    print(f"\n📈 Portfolio Average (Equal Weight):")
    print(f"   Average Return: {avg_return:.2%}")
    print(f"   Average Sharpe: {avg_sharpe:.2f}")
    print(f"   Average Max DD: {avg_drawdown:.2%}")
    
    print("\n✅ Analysis completed successfully!")
    print("💡 Ready for live trading implementation with proper risk management.")
else:
    print("⚠️ No results available for final summary")